<a href="https://colab.research.google.com/github/aabdulhakeem/Forward-and-backward-chaining/blob/main/forwardandbackward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
facts = {}
rules = {}

In [174]:
def convert_fact(line):
    fact = line.strip().split()
    if len(fact) > 1:
      if fact[1] == '>=' or fact[1] == '<=' or fact[1] == '<' or fact[1] == '>' or fact[1] == '=':
        return fact[0], (fact[1], int(fact[2]))
      else:
        return fact[0], fact[-1]

    elif len(fact):
        return fact[0], True
    else:
        return None, None

In [175]:
def add_fact(line, facts):
    var, val = convert_fact(line)
    facts[var] = val

In [176]:
with open('/content/facts.txt', 'r') as fact_file:
    for line in fact_file:
        add_fact(line, facts)

In [177]:
facts

{'seeds': ('=', 0),
 'diameter': ('=', 7),
 'skin_smell': True,
 'color': 'orange'}

In [178]:
def convert_rule(line):
    rule = line.strip().split(' THEN ')
    if_part = rule[0].strip()[3:]
    then_part = rule[1].strip()
    return if_part, then_part

In [179]:
def add_rule(line, i):
    if_part, then_part = convert_rule(line)
    rules[(if_part, i)] = then_part

In [180]:
with open('/content/rules.txt', 'r') as rule_file:
    i = 1
    for line in rule_file:
        add_rule(line, i)
        i += 1

In [181]:
rules

{('shape is long AND color is yellow', 1): 'fruit is banana',
 ('shape is round AND color is red AND size is medium', 2): 'fruit is apple',
 ('shape is round AND color is red AND size is small', 3): 'fruit is cherry',
 ('skin_smell', 4): 'perfumed',
 ('fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit',
  5): 'citrus_fruit',
 ('size is medium AND color is yellow AND perfumed', 6): 'fruit is lemon',
 ('size is medium AND color is green', 7): 'fruit is kiwi',
 ('size is big AND perfumed AND color is orange AND citrus_fruit',
  8): 'fruit is grapefruit',
 ('perfumed AND color is orange AND size is medium', 9): 'fruit is orange',
 ('perfumed AND color is red AND size is small AND seeds = 0',
  10): 'fruit is strawberry',
 ('diameter < 2', 11): 'size is small',
 ('diameter > 10', 12): 'size is big',
 ('diameter > 2 AND diameter < 10', 13): 'size is medium'}

In [182]:
def compare(a, op, b, op2):
  if op2 == '=':
    if op == '>=':
        return a >= b
    elif op == '<=':
        return a <= b
    elif op == '<':
        return a < b
    elif op == '>':
        return a > b
    elif op == '=':
        return a == b

In [183]:
def check_rule(if_part, facts):
    check_list = []
    And = False
    Or = False
    if 'AND' in if_part:
        And = True
        if_part = if_part.split(' AND ')
    elif 'OR' in if_part:
        Or = True
        if_part = if_part.split(' OR ')
    else:
        if_part = [if_part]

    for part in if_part:
        var, val = convert_fact(part)
        if isinstance(val, tuple):
            comp = compare(facts.get(var)[1], val[0], val[1], facts.get(var)[0])
            check_list.append(comp)
        else:
            if facts.get(var) == val:
                check_list.append(True)
            else:
                check_list.append(False)

    if And:
        if all(check_list):
            return True
        else:
            return False
    elif Or:
        if any(check_list):
            return True
        else:
            return False
    else:
        return check_list[0]

In [184]:
goal = 'citrus_fruit'
value = True

In [197]:
def forward_chaining(goal, value):
    cycle = 0
    rules_copy = rules.copy()
    facts_copy = facts.copy()
    print(f"before starting, initial facts: {facts_copy}\n")
    while rules_copy:
        cycle += 1
        fire = False
        for if_part in rules_copy:
            if check_rule(if_part[0], facts_copy):
                fire = True
                add_fact(rules_copy[if_part], facts_copy)
                i = if_part[1]
                print(f"cycle {cycle}, R{i}:({if_part[0]} -> {rules_copy[if_part]}) fired")
                print(f"facts: {facts_copy}\n")
                del rules_copy[if_part]
                break
        if facts_copy.get(goal) == value:
            print(f"cycle {cycle}, goal reached, facts: {facts_copy}")
            return True
            break
        elif not fire:
            print(f"cycle {cycle}, no rule fired, facts: {facts_copy}")
            return False
            break

In [198]:
forward_chaining(goal, value)

before starting, initial facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange'}

cycle 1, R4:(skin_smell -> perfumed) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', 'perfumed': True}

cycle 2, R13:(diameter > 2 AND diameter < 10 -> size is medium) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}

cycle 3, R9:(perfumed AND color is orange AND size is medium -> fruit is orange) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}

cycle 4, R5:(fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit -> citrus_fruit) fired
facts: {'seeds': ('=', 0), 'diameter': ('=', 7), 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange', 'citrus_fruit': True}

cycle 4, goal reached, facts: {'

True